In [1]:
import traceback
import warnings

from io import StringIO
import ee
import time
import os
from glob import glob
from typing import Optional, Callable, List, Tuple
from shapely.geometry import mapping, Polygon
import numpy as np
import geopandas as gpd
import pandas as pd
import fsspec
from datetime import datetime, timezone
import math
import json

from ml4floods.data import ee_download, utils


# Calling some functions

Removed the full function call and used the following instead.

In [2]:
get_count_function = ee_download.get_count_function

In [3]:
get_landsat_collection = ee_download.get_landsat_collection

In [4]:
collection_mosaic_day = ee_download.collection_mosaic_day

In [5]:
BANDS_NAMES = ee_download.BANDS_NAMES

Calling the aoi file

# Reading the geojson file

In [6]:
sydney_aoi= gpd.read_file('2022-07-21_aois.geojson')

In [7]:
sydney_aoi

,aoi_code,geometry
0,AOI01,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
1,AOI01,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
2,AOI02,"POLYGON ((147.09960 -31.04462, 147.30960 -31.0..."
3,AOI02,"POLYGON ((147.09960 -31.04462, 147.30960 -31.0..."
4,AOI0289,"POLYGON ((147.49960 -32.44461, 147.70960 -32.4..."
...,...,...
476,AOI97,"POLYGON ((150.29960 -34.24462, 150.50960 -34.2..."
477,AOI98,"POLYGON ((150.29960 -34.04462, 150.50960 -34.0..."
478,AOI98,"POLYGON ((150.29960 -34.04462, 150.50960 -34.0..."
479,AOI99,"POLYGON ((150.29960 -33.84462, 150.50960 -33.8..."


In [8]:
sydney_aoi.geometry

0      POLYGON ((147.09960 -31.24462, 147.30960 -31.2...
1      POLYGON ((147.09960 -31.24462, 147.30960 -31.2...
2      POLYGON ((147.09960 -31.04462, 147.30960 -31.0...
3      POLYGON ((147.09960 -31.04462, 147.30960 -31.0...
4      POLYGON ((147.49960 -32.44461, 147.70960 -32.4...
                             ...                        
476    POLYGON ((150.29960 -34.24462, 150.50960 -34.2...
477    POLYGON ((150.29960 -34.04462, 150.50960 -34.0...
478    POLYGON ((150.29960 -34.04462, 150.50960 -34.0...
479    POLYGON ((150.29960 -33.84462, 150.50960 -33.8...
480    POLYGON ((150.29960 -33.84462, 150.50960 -33.8...
Name: geometry, Length: 481, dtype: geometry

Getting the bounds from a single polygon

In [9]:
test_bounds= sydney_aoi.geometry[0].bounds
test_bounds

(147.09959654580408,
 -31.244618412374592,
 147.30959654580408,
 -31.03461841237459)

# Sorting the dates

Converting standard date format into a datetime objectget_landsat_collection(date_start= , date_end= , bounds= test_bounds)

In [10]:
date_string_start = '02/08/21'
date_string_end = '03/09/21'
datetime_start= datetime.strptime(date_string_start, '%d/%m/%y')
datetime_end= datetime.strptime(date_string_end, '%d/%m/%y')
datetime_start

datetime.datetime(2021, 8, 2, 0, 0)

In [11]:
date_range = ('02/08/21', '03/09/21')

In [12]:
type(date_range)

tuple

In [13]:
#Test of datetime function from query notebook

#e.g. date_range = ('dd/mm/yy', 'dd/mm/yy')

def get_datetime(date_range):
    datetime_start= datetime.strptime(date_range[0], '%d/%m/%y')
    datetime_end= datetime.strptime(date_range[1], '%d/%m/%y')
    
    start_year = datetime_start.year
    end_year = datetime_end.year
    
    return datetime_start, datetime_end, start_year, end_year

dates = get_datetime(date_range)

print(dates)

(datetime.datetime(2021, 8, 2, 0, 0), datetime.datetime(2021, 9, 3, 0, 0), 2021, 2021)


In [14]:
type(dates[0])

datetime.datetime

In [15]:
print(f'Start datetime: {dates[0]}, end datetime: {dates[1]}')

Start datetime: 2021-08-02 00:00:00, end datetime: 2021-09-03 00:00:00


# Authenticating `ee`

First will have to run `earthengine authenticate` in the VM command line / terminal. 

**Issue:** Doesn't allow us to authenticate. We get an 'access blocked' message. We checked the GCP API information under APIs and Servicse > Enabled APIs & Services and there are no API credentials (client ID and key). We had a look into what the issue might be and found that we may need to enable 'Google Earth Engine API', but this did nothing. 

- Do we need client ID and key for the API to get access?
- Do we need to open the authentication link  in a VM browser? 
- Any other reasons why the authentication link doesn't work


**SOLVED** Cormac suggested to run the following

In [16]:
ee.Authenticate()

Enter verification code:  4/1AdQt8qiwM6xAqCh8lFoQq1hlCK04dVjM6QupcEPogeDpjzTAayMUPD4CQbY



Successfully saved authorization token.


Initialise earth engine (ee)

In [17]:
ee.Initialize()

# Getting the bounds and getting the images

In [18]:
type(sydney_aoi.geometry[0].bounds)

tuple

In [19]:

js = json.loads(sydney_aoi.to_json()) # Converting the aoi to a json and loading the json information, inludes coordinates,
                                        # id for each polygon. Within the dictionary we have a 'featureCollection'
    
bounds = ee.Geometry(ee.FeatureCollection(js).geometry()) # Getting the bounds of the polygons 

type(bounds)

ee.geometry.Geometry

In [20]:
landsat_imgs = get_landsat_collection(date_start=dates[0], date_end=dates[1] , bounds=bounds)

In [21]:
type(landsat_imgs)

ee.imagecollection.ImageCollection

# Checking the properties of the imageCollection

Checking the size of the image collection

NOTE: Find date range limits that won't time out

**Example** from https://developers.google.com/earth-engine/apidocs/ee-imagecollection-size

In [22]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(ee.Geometry.Point(-90.70, 34.71)).filterDate('2020-01-01', '2021-01-01');

In [23]:
type(col)

ee.imagecollection.ImageCollection

[running the below code in this link gives the number of images](https://code.earthengine.google.com/820fcb75a86f86352d4ae1d62c4650fe)

**End Example**

After checking the function `_get_collection()` in the script, I noticed the variable `n_images` which seemed like 'number of images'. Using the same code from the script gives us: 

In [24]:
landsat_imgs.size().getInfo()

12

We can see the `CLOUD_SCORE` is contained in 

features: [] -> id: 'date', properties: {} -> CLOUD_SCORE

In [25]:
myDict = landsat_imgs.getInfo()

In [26]:
len(myDict['features'])

12

In [27]:
myDict['features'][0].get('id')

'2021-08-17'

In [28]:
myDict = landsat_imgs.getInfo()

In [29]:
myDict['features'][0].get('properties')['CLOUD_COVER']

3.48

In [30]:
cloudScore_landsat = np.array([])

for i in range(len(myDict['features'])):
    val = myDict['features'][i].get('properties')['CLOUD_COVER']
    cloudScore_landsat = np.append(cloudScore_landsat, val)
    print(i, val) 


print(f'Mean cloud score is {cloudScore_landsat.mean()}')

0 3.48
1 10.97
2 40.43
3 76.51
4 0.32
5 0.05
6 54.69
7 10.8
8 0.02
9 14.17
10 1.96
11 0.01
Mean cloud score is 17.784166666666668


Now we need to know what a cloud score looks like and if there are any other properties we want to consider from the imageCollection.

plot out on a spatial grid. 

Each patch may have multiple images with each of the cloud scores. 

geopandas table with whichever coordinates used. coords and cloud cover.
sleep statement after each point to see how the plot is being built. 

---

geoscape query and intersects. 

---

linking the keys for both satellites.

To access the information, the procses looks like this: `landsat_imgs.getInfo()['features'][my_index].get('properties')['my_feature']`. 

getInfo() is a dictionary, so we access the dictionary by calling one of the words in the dictionary being 'features'. This now becomes a list of the different images, so we can index into the image we want. We then use the get() function which again returns a dictionary we can access with one of the words in the dictionary to access the information. 

**Now getting the images for s2. Note, s2 has different properties to landsat**

In [31]:
get_s2_collection = ee_download.get_s2_collection

In [32]:
s2_imgs = get_s2_collection(date_start=dates[0], date_end=dates[1] , bounds=bounds)

In [33]:
s2_imgs.getInfo()['features'][0].get('properties')['CLOUD_COVERAGE_ASSESSMENT']

10.1582

Number of images

In [34]:
s2_imgs.size().getInfo()

26

In [35]:
myDict2 = s2_imgs.getInfo()

In [36]:
cloudScore_s2 = np.array([])

for i in range(s2_imgs.size().getInfo()):
    val = myDict2['features'][i].get('properties')['CLOUD_COVERAGE_ASSESSMENT']
    cloudScore_s2 = np.append(cloudScore_s2, val)
    print(i, val)
    
print(f'Mean cloud score is {cloudScore_s2.mean()}')

0 10.1582
1 18.4946
2 19.9737
3 57.2791
4 69.7648
5 0
6 1.937
7 6.9454
8 16.4855
9 0
10 0
11 29.3754
12 0
13 98.0081
14 0
15 0
16 90.0543
17 99.9021
18 0
19 51.3534
20 0
21 99.9965
22 99.9962
23 0.2658
24 0
25 0
Mean cloud score is 29.615003846153847


In [ ]:
myDict['features']

# Checking the shapefile data type 

In [38]:
import shutil

In [39]:
extract= shutil.unpack_archive('Declarations_19722_Merge.zip')

In [40]:
test= gpd.read_file('Declarations_unzip/Declarations_19722_Merge.shp')

In [41]:
type(test)

geopandas.geodataframe.GeoDataFrame

In [42]:
type(test['geometry'])

geopandas.geoseries.GeoSeries

In [43]:
type(test['geometry'][37])

shapely.geometry.multipolygon.MultiPolygon

# Putting useful information into a dataframe

* CLOUD_COVER
* polygon
* satellite image ID associated with the polygon
* date associated with the specific satellite image


We need to find out what other useful information is in the properties of the imageCollection 

Checking the number of polygons in `js` which comes from `sydney_aoi`

In [44]:
len(js['features'])

481

Can also be done with 

In [45]:
len(sydney_aoi)

481

Checking the `myDict['features'][0].get('properties')` dictionary, some interesting properties to explore are: 
* CLOUD_COVER
* cloud_probability
* DATE_ACQUIRED
* 'system:index' 

this is another date, one day after DATE_ACQUIRED.
* CLOUD_COVER_LAND
* COLLECTION_NUMBER

Checking the `myDict['features'][0].get('id')` dictionary has a date that is one day after DATE_acquired.


In [46]:
cloudScore_landsat

array([3.480e+00, 1.097e+01, 4.043e+01, 7.651e+01, 3.200e-01, 5.000e-02,
       5.469e+01, 1.080e+01, 2.000e-02, 1.417e+01, 1.960e+00, 1.000e-02])

# Extracting properties for individual polygons from original AOI

In [47]:
def get_imageCollections(bounds: ee.geometry.Geometry, dates: tuple): # returns imageCollection

    get_landsat_collection = ee_download.get_landsat_collection
    landsat_imgs = get_landsat_collection(date_start=dates[0], date_end=dates[1] , bounds= bounds)

    get_s2_collection = ee_download.get_s2_collection
    s2_imgs = get_s2_collection(date_start=dates[0], date_end=dates[1] , bounds= bounds)
    
    return landsat_imgs, s2_imgs

In [48]:
def get_datetime(date_range: tuple): # function returns a tuple containing datetime
    datetime_start= datetime.strptime(date_range[0], '%d/%m/%y')
    datetime_end= datetime.strptime(date_range[1], '%d/%m/%y')
    
    start_year = datetime_start.year
    end_year = datetime_end.year
    
    return datetime_start, datetime_end, start_year, end_year 

In [49]:
def read_aoi(my_gpd_aoi:str):
    my_aoi = gpd.read_file(my_gpd_aoi)
    
    aoi_codes = my_aoi['aoi_code']
    js = json.loads(my_aoi.to_json()) # convertin gthe aoi to a json and getting the 
    bounds = ee.Geometry(ee.FeatureCollection(js).geometry())
    
    return bounds, aoi_codes

In [50]:
def extract_properties(landsat_imgs, s2_imgs, init_bounds, aoi_code):
    
    coordinates = init_bounds['coordinates']
    
    landsat_dict = landsat_imgs.getInfo()
    s2_dict = s2_imgs.getInfo()
    
    n_landsat = len(landsat_dict['features'])
    print('Landsat images:', n_landsat)
    n_s2 = len(s2_dict['features'])
    print('s2 images:', n_s2)
    
    n_total = n_landsat + n_s2 
    
    properties_df = pd.DataFrame(index=np.arange(0, n_total), columns=['id', 'source', 'cloud_cover', 'percentage_valid_pixels', 'coordinates', 'aoi_code']) 
    
    
    for i in range(n_landsat):
        properties_df['id'][i] = landsat_dict['features'][i].get('id')
        properties_df['source'][i] = 'landsat'
        properties_df['cloud_cover'][i] = landsat_dict['features'][i].get('properties')['CLOUD_COVER']
        properties_df['percentage_valid_pixels'][i] = landsat_dict['features'][i].get('properties')['valids'] 
        properties_df['coordinates'][i] = coordinates
        properties_df['aoi_code'][i] = aoi_code
        #find area/co-ordinates associated with each image
        
    for i in range(n_s2):
        j = i + n_landsat
        
        properties_df['id'][j] = s2_dict['features'][i].get('id')
        properties_df['source'][j] = 's2'
        properties_df['cloud_cover'][j] = s2_dict['features'][i].get('properties')['CLOUD_COVERAGE_ASSESSMENT']
        properties_df['percentage_valid_pixels'][j] = s2_dict['features'][i].get('properties')['valids'] 
        properties_df['coordinates'][j] = coordinates
        properties_df['aoi_code'][j] = aoi_code
        #find area/co-ordinates associated with each image
    
    return properties_df

In [51]:
test_dates = ('02/08/21', '03/09/21')

test_datetime = get_datetime(test_dates)

In [52]:
sydney_aoi, sydney_aoi_codes = read_aoi('2022-07-21_aois.geojson')

sydney_aoi_codes

0        AOI01
1        AOI01
2        AOI02
3        AOI02
4      AOI0289
        ...   
476      AOI97
477      AOI98
478      AOI98
479      AOI99
480      AOI99
Name: aoi_code, Length: 481, dtype: object

In [53]:
sydney_aoi_all = sydney_aoi.getInfo()['coordinates']

sydney_aoi_all[0]

[[[147.09959654580408, -31.244618412374592],
  [147.30959654580408, -31.244618412374592],
  [147.30959654580408, -31.03461841237459],
  [147.09959654580408, -31.03461841237459],
  [147.09959654580408, -31.244618412374592]]]

In [54]:
len(sydney_aoi_all)

263

In [55]:
sydney_aoi_poly = ee.Geometry.Polygon(sydney_aoi_all[0])

type(sydney_aoi_poly)

ee.geometry.Geometry

In [56]:
len(sydney_aoi_all)

263

In [57]:
poly_landsat, poly_s2 = get_imageCollections(sydney_aoi_poly, test_datetime)

In [58]:
test_poly_df = extract_properties(poly_landsat, poly_s2, sydney_aoi_poly, sydney_aoi_codes[0])

Landsat images: 4
s2 images: 13


In [59]:
test_poly_df

,id,source,cloud_cover,percentage_valid_pixels,coordinates,aoi_code
0,2021-08-13,landsat,0.02,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
1,2021-08-29,landsat,14.17,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
2,2021-08-04,landsat,1.96,0.279306,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
3,2021-08-20,landsat,0.01,0.276128,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
4,2021-08-03,s2,3.0108,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
5,2021-08-05,s2,40.112,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
6,2021-08-08,s2,2.0378,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
7,2021-08-10,s2,0,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
8,2021-08-13,s2,0.0281,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01
9,2021-08-15,s2,5.4465,1,"[[[147.09959654580408, -31.244618412374592], [...",AOI01


In [60]:
test_poly_df['coordinates'][0]

[[[147.09959654580408, -31.244618412374592],
  [147.30959654580408, -31.244618412374592],
  [147.30959654580408, -31.03461841237459],
  [147.09959654580408, -31.03461841237459],
  [147.09959654580408, -31.244618412374592]]]

In [61]:
test_poly_df.shape

(17, 6)

**QUESTION** Previously we had 12 images for landsat and 26 images for s2, is the decrease we see above due to only selecting a single AOI from `sydney_aoi_codes`? 

`test_poly_df` Now has the properties from the iamges for a single polygon of an AOI, being from `sydney_aoi_all[0]`. We want to link all the AOI codes to each polygon. 

It seems like for each polygon we need to check each image and associate the AOI to that polygon.

# Getting the images for multiple polygons

In [62]:
type(sydney_aoi_poly)

ee.geometry.Geometry

Getting the first 20 polygons instead of all 263 (long processing times when getting the imageCollection. If you run on all 263 polygons, takes about 10 minutes.

In [63]:
sydney_aoi, sydney_aoi_codes = read_aoi('2022-07-21_aois.geojson')

# sydney_aoi_codes

In [64]:
sydney_aoi_all = sydney_aoi.getInfo()['coordinates']

sydney_aoi_all[0]

[[[147.09959654580408, -31.244618412374592],
  [147.30959654580408, -31.244618412374592],
  [147.30959654580408, -31.03461841237459],
  [147.09959654580408, -31.03461841237459],
  [147.09959654580408, -31.244618412374592]]]

In [65]:
sydney_aoi_poly = []
for i in range(len(sydney_aoi_all[0:20])):
    sydney_aoi_poly = np.append(sydney_aoi_poly, ee.Geometry.Polygon(sydney_aoi_all[i]))
    
type(sydney_aoi_poly)

numpy.ndarray

In [66]:
type(sydney_aoi_poly[0])

ee.geometry.Geometry

Getting the first 20 codes to match the first 20 polygons

In [67]:
sydney_aoi_codes = sydney_aoi_codes[0:20]

In [68]:
len(sydney_aoi_poly)

20

In [69]:
def get_imageCollections(bounds: np.ndarray, dates: tuple): # returns imageCollection

    get_landsat_collection = ee_download.get_landsat_collection
    get_s2_collection = ee_download.get_s2_collection
    
    landsat_imgs = []
    s2_imgs = []
    
    for i in range(len(sydney_aoi_poly)):
        landsat_imgs = np.append(landsat_imgs, get_landsat_collection(date_start=dates[0], date_end=dates[1] , bounds= bounds[i]))
        s2_imgs = np.append(s2_imgs, get_s2_collection(date_start=dates[0], date_end=dates[1] , bounds= bounds[i]))

    return landsat_imgs, s2_imgs

In [70]:
poly_landsat, poly_s2 = get_imageCollections(sydney_aoi_poly, test_datetime)

In [71]:
len(poly_landsat[0].getInfo()['features'])

4

Checking how many images are in each of the firs 20 polygons

In [72]:
for i in range(len(poly_landsat)):
    print(len(poly_landsat[i].getInfo()['features']))    

4
4
2
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [73]:
len(sydney_aoi_poly)

20

In [74]:
sydney_aoi_poly[0]['coordinates']

[[[147.09959654580408, -31.244618412374592],
  [147.30959654580408, -31.244618412374592],
  [147.30959654580408, -31.03461841237459],
  [147.09959654580408, -31.03461841237459],
  [147.09959654580408, -31.244618412374592]]]

In [84]:
landsat_coordinates = [[] for _ in range(len(poly_landsat))] # creating number of lists to match the number of polygons so that each polygon will have it's coordinates
n_landsat = []

for i in range(len(poly_landsat)):
    landsat_coordinates[i] = sydney_aoi_poly[i]['coordinates']

    n_landsat = np.append(n_landsat, len(poly_landsat[i].getInfo()['features'])) # Of the 20 indices, there will be the number of images in each. 


Trying to make a DF that isn't nested. Creating the DF comes later, first let's try to make lists that will be used to populate the DF. 

In [76]:
id_list = []
cloudCover_list = []
validPixels_list = []
# coordinates_list = []
# aoi_list = []
source_list = ['landsat']*np.int(n_landsat.sum()) # creating a list the size of the total number of images of all the polygons

for i in range(len(poly_landsat)): # Iterating through all the polygons
    
    for k in range(np.int(n_landsat[i])): # Iterating through all the images of each polygon
        
        id_list = np.append(id_list, poly_landsat[i].getInfo()['features'][k].get('id'))
        cloudCover_list = np.append(cloudCover_list, poly_landsat[i].getInfo()['features'][k].get('properties')['CLOUD_COVER'])
        validPixels_list = np.append(validPixels_list, poly_landsat[i].getInfo()['features'][k].get('properties')['valids'])
        # coordinates_list = landsat_coordinates[i]
        # aoi_list = sydney_aoi_codes[i]


Having issues with coordinates and aoi, commented them out and will work on them in a separate cell. 

In [78]:
len(landsat_coordinates)

20

In [85]:
landsat_coordinates[0]

[[[147.09959654580408, -31.244618412374592],
  [147.30959654580408, -31.244618412374592],
  [147.30959654580408, -31.03461841237459],
  [147.09959654580408, -31.03461841237459],
  [147.09959654580408, -31.244618412374592]]]

In [ ]:
coordinates_list = [] # creating number of lists to match the number of polygons so that each polygon will have it's coordinates

aoi_list = []

for i in range(len(poly_landsat)): # Iterating through all the polygons
    
    for k in range(len(poly_landsat[i].getInfo()['features'])): # Iterating through all the images of each polygon
        
        print(k)
        
        aoi_list = np.append(aoi_list, sydney_aoi_codes[i])
        
        index_coord = landsat_coordinates[i]
        
        print(index_coord)

        coordinates_list.extend(index_coord)

In [110]:
len(coordinates_list)

76

In [111]:
type(coordinates_list[0])

list

Managed to get the aoi list, but the coordinates list is having issues. For every polygon, the array of coordinates needs to be duplicated to the number of images that are in the polygon. 

In [114]:
landsat_gdf = gpd.GeoDataFrame({'id':id_list, 'cloud_cover':cloudCover_list, 'source': source_list, 'aoi_code': aoi_list, 'percentage_valid_pixels': validPixels_list, 'geometry':coordinates_list})
landsat_gdf.head(10)

,id,cloud_cover,source,aoi_code,percentage_valid_pixels,geometry
0,2021-08-13,0.02,landsat,AOI01,1.000000,"[[147.09959654580408, -31.244618412374592], [1..."
1,2021-08-29,14.17,landsat,AOI01,1.000000,"[[147.09959654580408, -31.244618412374592], [1..."
2,2021-08-04,1.96,landsat,AOI01,0.279306,"[[147.09959654580408, -31.244618412374592], [1..."
3,2021-08-20,0.01,landsat,AOI01,0.276128,"[[147.09959654580408, -31.244618412374592], [1..."
4,2021-08-13,0.02,landsat,AOI01,1.000000,"[[147.09959654580408, -31.04461841237459], [14..."
5,2021-08-29,14.17,landsat,AOI01,1.000000,"[[147.09959654580408, -31.04461841237459], [14..."
6,2021-08-04,1.96,landsat,AOI01,0.521599,"[[147.09959654580408, -31.04461841237459], [14..."
7,2021-08-20,0.01,landsat,AOI01,0.518510,"[[147.09959654580408, -31.04461841237459], [14..."
8,2021-08-13,2.36,landsat,AOI02,1.000000,"[[147.4996031608572, -32.44460560526437], [147..."
9,2021-08-29,56.97,landsat,AOI02,1.000000,"[[147.4996031608572, -32.44460560526437], [147..."


In [115]:
type(landsat_gdf.geometry[0])

list

In [116]:
from shapely.geometry import Polygon

landsat_gdf['geometry'] = landsat_gdf['geometry'].apply(Polygon)

In [117]:
landsat_gdf

,id,cloud_cover,source,aoi_code,percentage_valid_pixels,geometry
0,2021-08-13,0.02,landsat,AOI01,1.000000,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
1,2021-08-29,14.17,landsat,AOI01,1.000000,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
2,2021-08-04,1.96,landsat,AOI01,0.279306,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
3,2021-08-20,0.01,landsat,AOI01,0.276128,"POLYGON ((147.09960 -31.24462, 147.30960 -31.2..."
4,2021-08-13,0.02,landsat,AOI01,1.000000,"POLYGON ((147.09960 -31.04462, 147.30960 -31.0..."
...,...,...,...,...,...,...
71,2021-08-29,56.97,landsat,AOI0395,0.705777,"POLYGON ((148.29960 -32.24461, 148.50960 -32.2..."
72,2021-08-06,54.69,landsat,AOI0396,1.000000,"POLYGON ((148.29960 -32.04461, 148.50960 -32.0..."
73,2021-08-22,10.80,landsat,AOI0396,1.000000,"POLYGON ((148.29960 -32.04461, 148.50960 -32.0..."
74,2021-08-13,2.36,landsat,AOI0396,0.932296,"POLYGON ((148.29960 -32.04461, 148.50960 -32.0..."


In [128]:
landsat_gdf.groupby('geometry').sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  """Entry point for launching an IPython kernel.


TypeError: '<' not supported between instances of 'Polygon' and 'Polygon'

TypeError: '<' not supported between instances of 'Polygon' and 'Polygon'